In [7]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
%config Completer.use_jedi = False
import utils.db as db
import glob
import datetime
import folia.main as folia
from utils import db
mongo = db.get_db()


1. Download a fragment
2. Check if USC
3. If so get all transcripts
4. Get a word sequence from it
5. Find it in the transcripts

In [8]:
def get_all_fragments():
    mongo = db.get_db()
    results = mongo.fragments.find({},{'_id':0})
    results = [element for element in results]
    all_mid_nodes =[]
    all_fragments = []
    for result in results:
        mid_nodes = [element for element in result['tree']['children'] if len(element['children'])>0]
        for mid_node in mid_nodes:

            all_mid_nodes.append(mid_node['label'])
            for fragment in mid_node['children']:
                all_fragments.append({'fragment':fragment,'mid_label':mid_node['label'],'top_label':result['tree']['label']})
            
    return (all_fragments)

In [9]:
def update_fragments(main_node_label,mid_node_label,fragment,attributes):
    results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
    mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
    for x,element in enumerate(results['tree']['children']):
        if element['label']==mid_node_label:
            for n,leaves in enumerate(element['children']):
                if leaves['label'] == fragment['label']:
                    for attr in attributes:
                        results['tree']['children'][x]['children'][n][list(attr.keys())[0]]=attr[list(attr.keys())[0]]
                    
    replace = mongo.fragments.replace_one(mongo_id,results)


In [10]:
#update_fragments("rape","kill",fragment['fragment'],[{'media_index':100}])

In [11]:
all_fragments = get_all_fragments()

In [12]:
for fragment in all_fragments:
    if 'usc' in fragment['fragment']['testimony_id'].split('_'):
    #if 'usc_shoah_14900' in fragment['fragment']['testimony_id']:
        testimony_id = fragment['fragment']['testimony_id']
        sentence_start = fragment['fragment']['start_sentence_index']
        p1 = minidom.parse("Data/folia_output/"+testimony_id+'.xml')
        sentences = p1.getElementsByTagName('s')
        src_attr = sentences[int(sentence_start)].parentNode.getAttribute('src')
        part= src_attr.split('_')[0]
        time = src_attr.split('_')[1]
        update_fragments(fragment['top_label'],fragment['mid_label'],fragment['fragment'],[{'media_index':int(part)-1},{'media_offset':int(time)}])      

In [13]:
type(time)

str